In [ ]:
# %%capture
# !git clone https://github.com/nikitakapitan/nlphub.git
# !mkdir logs
# !pip install datasets transformers evaluate accelerate

# %cd nlphub
# !pip install .

In [ ]:
import yaml
import json
from datasets import load_dataset
from transformers import pipeline
from nlphub import ClassificationBenchmark

TASK = "text-classification"
BenchmarkClass = ClassificationBenchmark
DATASET_NAME = "imdb"
MODEL_NAME = "distilbert-base-uncased"

GLUE_TASKS = ["cola", "mnli", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]
METRICS = [ {'accuracy'   : {}},
            {'f1'         : {'average': 'weighted'}},
            {'glue'       : GLUE_TASKS}] 

In [ ]:
dataset = load_dataset(DATASET_NAME, split='test')

# truncation : crop input text to model max_length | device=0 : first available GPU
pipe = pipeline(TASK, model=MODEL_NAME, truncation=True, device=0) 
benchmark = BenchmarkClass(pipe, dataset, METRICS)
metrics = benchmark.run_benchmark()

with open('results.json', 'w') as f:
    setup = f"Compute {METRICS} for task={TASK} on dataset={DATASET_NAME} using model={MODEL_NAME}"
    json.dump({'setup':setup, 'metrics' : metrics}, f, indent=4)